Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# CSE204 - Introduction to Machine Learning - Lab Session 6: Building a Neural Network From Scratch - feedforward

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

[CSE204-2021](https://moodle.polytechnique.fr/course/view.php?id=12838) Lab session #06

J.B. Scoggins - Adrien Ehrhardt

## Introduction

In this lab you will learn to create a simple multilayer perceptron (MLP) neural network from scratch using only the Numpy package for matrix-vector operations.  In order to train the network on several datasets, you will need to implement the back-propagation algorithm with gradient descent (in lab 07!). Before getting started, let's review the notations we will use in this lab and recall the stochastic gradient descent algorithm.

### Notation

We will consider simple feed-forward networks that can be described by the following recursive relationship,

$$
\begin{align}
&z^l = a^{l-1} W^l + b^l,\\
&a^l = \sigma^l(z^l),
\end{align}
$$

where $a^l$ is the output (activation) of layer $l$ which is a nonlinear function $\sigma^l$ of a linear transformation of the previous layer's output.  The linear transformation is performed using the weight matrix $W^l$ and bias vector $b^l$ associated with the layer $l$.  We will denote the last layer in the network with a capital $L$ superscript.  The recursion is stopped by setting $a^0 = x$, where $x$ is the input vector to our network.  Note that in the recursive expressions above, we implicitly assume that our input/output vectors are row vectors.  The reason for this will be apparent later.

Taking this notation into account, we see that a network with $L-1$ hidden layers is fully expressed by its $L$ weight matrices, bias vectors, and activation functions.  We can denote the set of trainable parameters in our network by $\theta = \{ W^1, \dots, W^L, b^1, \dots, b^L \}$.

In this lab, we are only concerned with supervised learning tasks.  Recall that in supervised learning, we have a dataset represented by a list of $(x, y)$ pairs where $x$ is the input to our model and $y$ is the desired output.

- For regression problems where we want to fit a function $y = f(x)$, $x$ is the independent variable vector, and $y$ is the function value.
- In classification problems, $x$ will correspond to a set of attributes and $y$ the corresponding label.

The goal of supervised learning is to "train" our network by adjusting its parameters in order to minimize a cost function over the entire training set,

$$
\min_\theta \mathcal{L} = \min_\theta \sum_{i=1}^N \ell^{(i)}
$$

where $\ell^{(i)} = \ell(\sigma^L(\sigma^{L-1}(\dots(x^{(i)}W^1 + b^1)\dots)W^L + b^L), y^{(i)})$ and $\ell(\hat{y}, y)$ denotes the particular form of the loss function being considered.  In this lab, we will use 2 different loss functions:

1. Quadratic Loss: $\ell(\hat{y}, y) = \|\hat{y} - y\|^2$
2. Cross-entropy Loss: $\ell(\hat{y}, y) = -[ y \ln\hat{y} + (1-y)\ln(1-\hat{y})]$

## Step 1: Build an untrainable network

Understanding (and implementing) the back-propagation algorithm can seem a little daunting at first. Therefore, let's start by building out the functions we need just to create a network that cannot be trained, but can predict (feedforward pass). First load the libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

*Python hints:* 
- Static methods are callable with uninstanciated class objects.
- `__call__` methods are retrieved when using parentheses with an instanciated class object.
- However, instanciation has precedence over `__call__`, so calling a Class like an other static method of that class or a function will not work (see [here](https://stackoverflow.com/questions/26793600/decorate-call-with-staticmethod)).
- `__init__` (typically used to customize the instanciated objects of a class) cannot return anything.
- The trick is thus to use it in conjunction with `__new__` (which creates the object and passes it to `__init__`).

In [ ]:
class Activation(object):
    """
    Base activation node class
    """

    def __new__(self, x: float) -> float:
        """
        What to do when instantiating an object: this basically says Activation(x)
        will be equivalent to Activation()(x).

        :param float or np.array x: input of the activation
        :return: call the activation function
        :rtype: float or np.array
        """
        return self.__call__(x)

### Exercise 1: Implement the activation functions.

1. The easiest activation we can implement is the identity function which simply returns the input as itself.  Implement this below in the class template `Identity`. The `prime` function should implement the derivative of the activation. Note that it should work for single values (`float`) and vectors (`np.array`).

*Python hint:* see [`np.ones_like`](https://numpy.org/doc/stable/reference/generated/numpy.ones_like.html).

In [ ]:
class Identity(Activation):
    """
    Identity activation node
    """
    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the identity activation function

        :param float or np.array x: input of the activation
        :return: identity of input
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        
    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the identity activation function

        :param float or np.array: input of the activation
        :return: derivative of input
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
assert Identity(1) == 1
print(Identity(np.array([1, 1])))

2. The threshold activation function takes an input and returns 1 if the input is positive, otherwise 0.  Implement the `Threshold` class below. Note that it should work for single values (`float`) and vectors (`np.array`).

*Python hint:* see [`np.where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html), and [`np.zeros_like`](https://numpy.org/doc/stable/reference/generated/numpy.zeros_like.html).

In [ ]:
class Threshold(Activation):
    """
    Threshold activation node
    """
    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the threshold activation function

        :param float or np.array x: input of the activation
        :return: if x > 0, 1, else 0
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()
    
    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the threshold activation function

        :param float or np.array: input of the activation
        :return: derivative of input
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
assert Threshold(0.5) == 1
print(Threshold(np.array([-0.1, 0.1])))

3. Recall that the sigmoid function is given by $\sigma(x) = \dfrac{1}{1 + \exp(-x)}$. Calculate its derivative.

*Hint*: you can do this on paper but be sure to insert a new markdown cell below and then click Edit > Insert Image.

YOUR ANSWER HERE

4. Implement the `Sigmoid` class below. Note that it should work for single values (`float`) and vectors (`np.array`).

In [ ]:
class Sigmoid(Activation):    
    """
    Threshold activation node
    """
    @staticmethod
    def __call__(x: float) -> float:
        """
        Implements the sigmoid activation function

        :param float or np.array x: input of the activation
        :return: sigmoid transform of input
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

    @staticmethod
    def prime(x: float) -> float:
        """
        Implements the derivative of the sigmoid activation function
        (you can make good use of Sigmoid(...)!)

        :param float or np.array: input of the activation
        :return: derivative of input
        :rtype: float or np.array
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
assert Sigmoid(0) == 0.5
print(Sigmoid(np.array([-1, 0, 1])))

4. Plot the activation functions and their derivative on the domain [-5, 5] with the function below. Are the graphical results satisfactory? (no answer expected).

In [ ]:
def plot_activation(activation: Activation):
    x = np.arange(-5, 5, 0.01)
    plt.plot(x, activation(x))
    plt.plot(x, activation.prime(x))
    plt.legend((activation.__name__, activation.__name__ + " prime"))
    plt.show()

In [ ]:
plot_activation(Identity)
plot_activation(Threshold)
plot_activation(Sigmoid)

### Exercise 2: First draft of the Network class

During this lab, we will build on the following python class called `Network` which will represent our neural net.  The `Network` class will keep track of the weights, biases, and activations needed to evaluate and train our model.  The following exercises will guide you through building up the class from the skeleton below.

In [ ]:
class Network:
    """
    A simple implementation of a feed-forward artificial neural network.
    Work on this class throughout the entire exercise, rerunning this cell after each update.
    """

    def __init__(self, sizes: list, activations: list):
        """
        Construct a network given a list of the number of neurons in each layer and 
        a list of activations which will be applied to each layer.
        
        :param list sizes: A list of integers representing the number of nodes in each layer, 
        including the input and output layers.
        :param list activations: A list of callable objects representing the activation functions.
        Its size should be one less than sizes.
        """
        # Exercise 2.2
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def num_params(self) -> int:
        """Returns the total number of trainable parameters in this network."""
        # Exercise 2.3
        # YOUR CODE HERE
        raise NotImplementedError()
    
    
    def feed_forward(self, x: np.array) -> np.array:
        """
        Evaluates the network for the given input and returns the result.
        
        :param numpy.array x: A numpy 2D-array where the columns represent input variables
        and rows represent independent samples.
        :return: output of the network
        :rtype: numpy.array
        """
        # Exercise 2.4
        # YOUR CODE HERE
        raise NotImplementedError()

1. From the recursion formulas above, write down the shapes of the weight matrices and bias vectors, given the number of neurons $d^l$ in layer $l$.

YOUR ANSWER HERE

2. Implement the init `__init__` function in `Network` above.

* Construct a list of weight matrices (in a `weights` attribute), bias vectors (in a `biases` attribute) which are randomly initialized with normal distributions (see `np.random.randn`);

* store `sizes` and `activations` in attributes `sizes` and `sigmas` respectively.

*Python hints*: `array[1:]` gives you all the array except the first entry; `array[:-1]` gives you all the array except the last entry; `for x, y in zip(a, b)` lets you iterate "simultaneously" through a pair of objects of same length.

In [ ]:
# Test the network's initialization
assert Network([1], Sigmoid).sizes == [1]

3. Implement the `num_params` function in `Network` above.  Use your knowledge of the shapes of the weights and biases.

In [ ]:
# Test the network's number of parameters
assert Network([1, 2, 2, 1], [Sigmoid, Sigmoid, Sigmoid]).num_params() == 13

4. Implement the `feed_forward` function in `Network` above. Use the recurance relations discussed in the Notation section, in particular:

$$z^l = a^{l-1} W^l + b^l,\\
a^l = \sigma^l(z^l)$$

In [ ]:
# Test the network's feedforward result
assert Network([1, 2, 2, 1], [Sigmoid, Sigmoid, Sigmoid]).feed_forward(np.array([1])).shape[0] == 1

5. Create a `network` with 2 hidden layers of 5 nodes each that takes 4 inputs and has 1 output (use any activation).

In [ ]:
# network = ...  # <- TO UNCOMMENT AND COMPLETE
# YOUR CODE HERE
raise NotImplementedError()

6. Print the weights and biases of the network and confirm they are intialized correctly (shape and values).

In [ ]:
for index, weight in enumerate(network.weights):
    print("\nWeight", index)
    print("Shape:", weight.shape)
    print(weight)

In [ ]:
for index, bias in enumerate(network.biases):
    print("\nBias", index)
    print("Shape:", bias.shape)
    print(bias)

### Exercise 4: Build logic gates

Below is a table of logical functions (logic gates).  Each function takes two values (A and B) representing True (1) or False (0) propositions and returns a True or False value.

| A | B | AND | OR | XOR | NAND |
|---|---|-----|----|-----|------|
| 0 | 0 | 0   | 0  | 0   | 1    |
| 0 | 1 | 0   | 1  | 1   | 1    |
| 1 | 0 | 0   | 1  | 1   | 1    |
| 1 | 1 | 1   | 1  | 0   | 0    |

Interestingly, [it is possible to create any boolean function of any size through a combination of NAND gates](https://en.wikipedia.org/wiki/NAND_gate)!  Thus, if we can create a network which reproduces the logic behind a NAND-gate, it is possible to represent any logical function (and by extension any mathematical function) by combining such a network into ever more complex networks.  This is one version of a universal approximation theorem for ANNs.

**Task:** build the AND, OR, and NAND logic gates above using the simple network below by **modifying its weights and biases directly**. The `logic_gate` function is provided to test your networks: it corresponds to columns A, B in the table above. We will test if we get the expected results (columns AND, ..., NAND).

In [ ]:
def logic_gate(network: Network, array: np.array = np.asarray(
            [[0, 0],
            [0, 1],
            [1, 0],
            [1, 1]])) -> np.array:
    """
    Helper function to test our network as a logic gate.

    :param Network network: our neural network
    :param numpy.array array: apply feed forward to array
    :return: output of network given array
    :rtype: numpy.array
    """
    return network.feed_forward(array)

In [ ]:
# Create a simple "logic gate" network with two inputs A and B and 1 output, with activation function Threshold
network = Network([2, 1], [Threshold])

1. Implement the AND gate by modifying the `network`'s weights and biases.

*Hint:* what should $W$ and $b$ be such that we get `[0, 0, 0, 1]` when the input is (A, B)?

In [ ]:
# AND gate
# network.weights[0] =  ...
# network.biases[0] = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test if we get the expected result (see table above)
np.testing.assert_array_equal(logic_gate(network), np.array([0, 0, 0, 1]).reshape((-1, 1)))

2. Implement the OR gate by modifying the `network`'s weights and biases.

In [ ]:
# OR gate
# network.weights[0] = ...
# network.biases[0] = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test if we get the expected result (see table above)
np.testing.assert_array_equal(logic_gate(network), np.array([0, 1, 1, 1]).reshape((-1, 1)))

3. Implement the NAND gate by modifying the `network`'s weights and biases.

In [ ]:
# NAND gate
# network.weights[0] = ...
# network.biases[0] = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test if we get the expected result (see table above)
np.testing.assert_array_equal(logic_gate(network), np.array([1, 1, 1, 0]).reshape((-1, 1)))

## Step 2: Training the network

While interesting, the `Network` class above is pretty useless as it stands since there is no way to learn a function we don't already know.  In this step, we will add the ability to train our network on a dataset using the gradient descent and back-propagation algorithms.  Let's review both of these algorithms now.

### (Stochastic) Gradient Descent

Recall from the beginning of the lab that we want to train the network parameters by minimizing a given loss function over an entire dataset. One method of doing this is using the gradient descent (GD) algorithm which you have already seen in the lectures and previous labs; recall the update rule:

$$
\theta = \theta - \eta \nabla_\theta \mathcal{L},
$$

where $\eta$ is the learning rate.  The update rule above is called GD because direction of change in the network parameters follow the opposite of the parameters' gradient in the loss function.  You can think of this like a ball rolling down a hill to find the minimum of the topology. Only in this case, the ball is massless because it has no momentum. Note that, contrary to linear and logistic regression, the loss / error function might not be convex, hence the existence of a unique minimizer isn't guaranteed, nor is it guaranteed that gradient descent will find it.

When the update rule is applied to a random subset of the total dataset, it is called stochastic gradient descent (SGD). SGD is far more efficient (computationally) than GD when the batch size is large enough to approximate the true gradients while being significantly smaller than the full dataset. Running over the entire dataset with SGD once is called an "epoch" (of training).

### Back-Propagation

From the GD update rule above, it is clear we will need to compute the gradients of the network parameters with respect to the cost function. This is exactly what back-propagation does, and thus is a crucial component to almost all neural network learning algorithms. In the next exercise, we will derive the 4 equations in back-propagation. You will need knowledge of the [chain rule for differentiation](https://en.wikipedia.org/wiki/Chain_rule) if you are not already familiar with this.

### Exercise 5: Derive backprop formulas

Before we start, it is convenient to define the following variable:

$$
\delta^l \equiv \frac{\partial \ell_p}{\partial z^l}.
$$

In other words, $\delta^l$ is the gradient of the loss function for a point $p$ with respect to the input to the activation function for the layer $l$ in our network.

1. What is the shape of $\delta^L$?

YOUR ANSWER HERE

2. Show that $\delta^L = \nabla_{a^L} \ell_p  \odot {\sigma'}^L ( z^L )$.

*Hint:* apply the chain rule to the definition of $\delta^L$.

YOUR ANSWER HERE

3. Show that for $l < L$, $\delta^l = [\delta^{l+1} (W^{l+1})^T ] \odot {\sigma'}^l ( z^l )$.

YOUR ANSWER HERE

4. Show that $\nabla_{W^l}\ell_p = (a^{l-1})^T \delta^l$.

*Hint:* use the definition of $z^l$ and derive w.r.t. a single component $W_{i,j}$.

YOUR ANSWER HERE

5. Show that $\nabla_{b^l}\ell_p = \delta^l$.

YOUR ANSWER HERE

Note that these four last formulas allow us to compute the gradient of the loss function for a single training point with respect to the parameters of our network.  Looking at the equations more closely, you should see a possible algorithm form.

1. Compute the values of $z^l$ and $a^l$ for $l = {1, \dots, L}$ by forward propagation through the network.  Recall $a^0 = x_p$: that's what we've implemented throughout Exercises 1-4.
2. Use the equation in Exercise 5.2 above to compute $\delta^L$.  
3. Back-propagate in the reverse direction using the equation in Exercise 5.3 to get all the other $\delta^l$ values.
4. Compute the parameter gradients using the equations in Exercises 5.4 and 5.5.

Note that this will yield the gradients for a single training point.  The gradients for the total loss function can easily be computed by

$$
\nabla_\theta \mathcal{L} = \sum_{p=1}^N \nabla_\theta \ell_p.
$$

Of course, the exact form of $\nabla_\theta \ell_p$ will depend on the choice of $\ell$.

We will implement this algorithm next week.